In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



Hipparcos Catalog Cluster Visualization



Goal: Apply DBSCAN and plot data with and without outliers in order to identify clusters. Then they can be compared to known clusters

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

df_full = pd.read_csv("/kaggle/input/hipparcos-star-catalog/hipparcos-voidmain.csv")
print(df_full.columns)
df = df_full[df_full["Plx"]>0]
PAR = np.array(df["Plx"],dtype=float)
RA = np.array(df["RAdeg"],dtype=float)
DEC = np.array(df["DEdeg"],dtype=float)
m = np.array(df["Vmag"],dtype=float)
bmv = np.array(df["B-V"],dtype=float)
RA *= np.pi/180 #convert to radians
DEC *= np.pi/180 #convert to radians

In [ ]:
# Anaconda code 

Distance = 1/PAR*1000   # parallax in database is in milli arcseconds

Phi = (np.pi/2-DEC)
Theta = RA

x = Distance*np.cos(Theta)*np.sin(Phi)
y = Distance*np.sin(Theta)*np.sin(Phi)
z = Distance*np.cos(Phi)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x,y,z,'.')
plt.show()

In [ ]:
# x y and z into a single array 
xyz = np.transpose(np.array([x,y,z]))

In [ ]:
# DBSCN clustering algorithm 

from sklearn.cluster import DBSCAN
db=DBSCAN(eps=2,min_samples=4,metric='euclidean')


In [ ]:
model=db.fit(xyz)

In [ ]:
label=model.labels_

In [ ]:
df = pd.DataFrame({"RA":RA,"DEC":DEC,"label":label,"bmv":bmv})
filtered = df[df["label"]>-1]
import plotly.express as px
fig = px.scatter(filtered,x="RA",y="DEC",color="bmv")
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df,x="RA",y="DEC",color="label")
fig.show()

In [ ]:
from sklearn import metrics

# identifying the points which makes up core points
sample_cores=np.zeros_like(label,dtype=bool)

sample_cores[db.core_sample_indices_]=True

# calculating the number of clusters

n_clusters=len(set(label))- (1 if -1 in label else 0)
print('No of clusters:',n_clusters)
